In [3]:
import pandas as pd
import json

# Load the CSV file
csv_file_path = 'gdf_filtered_hospital.csv'  # Replace with your CSV file path
csv_data = pd.read_csv(csv_file_path)

# Extract unique CTUID values from the CSV file
csv_ctuids = set(csv_data['CTUID'])

# Load the JSON file
json_file_path = 'toronto_ct_boundaries_4326.json'  # Replace with your JSON file path
with open(json_file_path, 'r') as f:
    json_data = json.load(f)

# Extract unique CTUID values from the JSON file
json_ctuids = set(feature['properties']['CTUID'] for feature in json_data['features'])

# Find CTUIDs that are in CSV but not in JSON
csv_not_in_json = csv_ctuids - json_ctuids

# Find CTUIDs that are in JSON but not in CSV
json_not_in_csv = json_ctuids - csv_ctuids

# Print the results
if not csv_not_in_json and not json_not_in_csv:
    print("All CTUIDs match 1:1 between CSV and JSON files.")
else:
    if csv_not_in_json:
        print("CTUIDs in CSV but not in JSON:", csv_not_in_json)
    if json_not_in_csv:
        print("CTUIDs in JSON but not in CSV:", json_not_in_csv)


CTUIDs in CSV but not in JSON: {5350346.01, 5350346.02, 5350349.0, 5350350.0, 5350353.02, 5350354.0, 5350355.05, 5350355.06, 5350355.02, 5350356.0, 5350357.01, 5350358.01, 5350359.0, 5350360.0, 5350363.07, 5350363.06, 5350365.0, 5350366.0, 5350367.01, 5350368.01, 5350368.02, 5350369.0, 5350370.02, 5350370.03, 5350371.0, 5350372.0, 5350373.0, 5350374.01, 5350375.01, 5350375.02, 5350375.03, 5350376.15, 5350376.16, 5350376.01, 5350376.02, 5350376.11, 5350377.03, 5350377.04, 5350377.06, 5350377.07, 5350376.12, 5350378.06, 5350378.08, 5350378.07, 5350378.04, 5350378.05, 5350378.11, 5350378.28, 5350378.23, 5350378.24, 5350378.25, 5350063.03, 5350063.04, 5350064.0, 5350065.01, 5350010.02, 5350013.02, 5350029.0, 5350030.0, 5350031.0, 5350032.0, 5350033.0, 5350034.01, 5350035.0, 5350036.0, 5350037.0, 5350046.0, 5350047.04, 5350047.02, 5350050.03, 5350050.04, 5350001.0, 5350002.0, 5350003.0, 5350004.0, 5350005.0, 5350006.0, 5350007.02, 5350008.02, 5350007.01, 5350008.01, 5350011.02, 5350012.03, 